In [12]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-5-mini")

messages = [
 HumanMessage("안녕 잘지냈어?") 
]

result = model.invoke(messages)
print(result)

content='안녕! 잘 지냈어. 너는 어떄? 궁금한 거나 도와줄 일 있으면 말해줘.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 229, 'prompt_tokens': 13, 'total_tokens': 242, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7c25Afw44OHHczdNCW5GD8FZmGp', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b7881-e747-7e02-8632-2b0261996aa6-0' usage_metadata={'input_tokens': 13, 'output_tokens': 229, 'total_tokens': 242, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}}


In [13]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool
def get_current_time(timezone: str, location: str) -> str:
  """ 현재 시각을 반환하는 함수

  Args:
    timezone: 타임존 (예: 'Asia/Seoul'). 실제 존재 해야함.
    location: 지역명. 타임존은 모든 지명에 대응되지 않으므로 이후 llm 답변 생성에 사용됨
  """
  
  tz = pytz.timezone(timezone)
  now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
  location_and_local_time = f"{timezone} ({location})의 현재 시각 {now}"
  
  print(location_and_local_time)
  return location_and_local_time

In [14]:
tools = [get_current_time]
tool_dict = {
  "get_current_time": get_current_time
}

llm_with_tools = model.bind_tools(tools)


In [15]:
from langchain_core.messages import SystemMessage

messages = [
  SystemMessage(content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
  HumanMessage(content="부산의 현재 시각은 몇인가요?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)



[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산의 현재 시각은 몇인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 212, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7cEPPdpGEpxi6jLdvqUPAuJ1anC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7882-1478-7681-b043-191f92d58913-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_p2oGCzsUCRbcZycp0HWt0jjQ', 'type': 'tool_call'}], usa

In [16]:
for tool_call in response.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)
  
messages


{'timezone': 'Asia/Seoul', 'location': 'Busan'}
Asia/Seoul (Busan)의 현재 시각 2026-01-01 16:42:33


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산의 현재 시각은 몇인가요?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 212, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7cEPPdpGEpxi6jLdvqUPAuJ1anC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7882-1478-7681-b043-191f92d58913-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_p2oGCzsUCRbcZycp0HWt0jjQ', 'type': 'tool_call'}], u

In [18]:
llm_with_tools.invoke(messages)

AIMessage(content='부산의 현재 시각은 2026-01-01 16:42:33 (대한민국 표준시, KST)입니다. 원하시면 다른 형식이나 다른 도시 시각도 알려드릴게요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 251, 'prompt_tokens': 274, 'total_tokens': 525, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7d0nqhE4IVS3yZEFMnBNGJfK5Bw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b7882-ce06-7090-9ae6-f79443983a8c-0', usage_metadata={'input_tokens': 274, 'output_tokens': 251, 'total_tokens': 525, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 192}})

In [19]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
  ticker: str = Field(..., title="주식 종목 코드", description="주식 종목 코드(예: 'AAPL')")
  period: str = Field(..., title="조회 기간", description="조회 기간 (예: '1d', '1mo', '1y')")

In [20]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
  """ 주식 종목 코드와 기간에 따른 주식 시가총액 조회 """
  
  ticker = yf.Ticker(stock_history_input.ticker)
  history = ticker.history(period=stock_history_input.period)
  return history.to_markdown()

tools = [get_current_time, get_yf_stock_history]
tool_dict = {
  "get_current_time": get_current_time,
  "get_yf_stock_history": get_yf_stock_history
}

llm_with_tools = model.bind_tools(tools)

In [21]:
messages.append(HumanMessage(content="테슬라는 한달전에 비해 주가가 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)
print(messages)
messages.append(response)


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산의 현재 시각은 몇인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 212, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7cEPPdpGEpxi6jLdvqUPAuJ1anC', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7882-1478-7681-b043-191f92d58913-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': 'Busan'}, 'id': 'call_p2oGCzsUCRbcZycp0HWt0jjQ', 'type': 'tool_call'}], usa

In [22]:
for tool_call in response.tool_calls:
  selected_tool = tool_dict[tool_call["name"]]
  print(tool_call["args"])
  tool_msg = selected_tool.invoke(tool_call)
  messages.append(tool_msg)
  print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2025-12-01 00:00:00-05:00 | 425.32 | 433.66 | 425.29 |  430.14 | 5.74636e+07 |           0 |              0 |\n| 2025-12-02 00:00:00-05:00 | 430.81 | 436.8  | 422.12 |  429.24 | 6.93366e+07 |           0 |              0 |\n| 2025-12-03 00:00:00-05:00 | 432.1  | 447.92 | 431.11 |  446.74 | 8.7483e+07  |           0 |              0 |\n| 2025-12-04 00:00:00-05:00 | 449.94 | 454.63 | 445.39 |  454.53 | 7.19065e+07 |           0 |              0 |\n| 2025-12-05 00:00:00-05:00 | 453.03 | 458.87 | 451.66 |  455    | 5.64275e+07 |           0 |              0 |\n| 2025-12-08 00:00:00-05:00 | 447.45 | 449.75 | 435.25 |  439.58 | 6.91658e+07 |           0 |              0 |\n| 2025-12-09 00:00:00-05:0

In [23]:
llm_with_tools.invoke(messages)

AIMessage(content='올랐습니다.\n\n- 2025-12-01 종가: $430.14  \n- 2025-12-31 종가: $449.72  \n- 상승폭: +$19.58 (약 +4.6%)\n\n(데이터는 제공된 1개월 일별 종가 기준이며 투자 판단은 본문 정보를 포함해 신중히 하시기 바랍니다.)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 413, 'prompt_tokens': 1737, 'total_tokens': 2150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Ct7lQBO5ytM6T8BF6dh9rgUQoW4zv', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b788a-cb31-7350-8d76-71f417c5c45b-0', usage_metadata={'input_tokens': 1737, 'output_tokens': 413, 'total_tokens': 2150, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 320}})

In [25]:
for c in model.stream([HumanMessage(content="잘지냈어? 한국 사회의 문제점이 무엇인지 얘기해줘")]):
  print(c.content, end="|")

|나|도| 잘| 지|냈|어| —| 물|어|줘|서| 고|마|워|!| 한국| 사회|의| 문제|점|에| 대해| 전|반|적으로| 정|리|해|줄|게|.| 주요| 이|슈|들을| 간|단|히| 짚|고|,| 왜| 문제|인지|와| 개선| 방향|도| 덧|붙|일|게|.| 특정| 분야|(|예|:| 청|년|,| 부|동|산|,| 젠|더| 등|)에| 더| 깊|게| 보고| 싶|으면| 말|해|줘|.

|주|요| 문제|와| 설명|
|-| 저|출|산|·|고|령|화|:| 출|산|율|이| 세계| 최|저|권|이고| 인|구|가| 빠|르게| 고|령|화|되고| 있어| 노동|력|,| 연|금|·|복|지| 재|정|,| 의료|·|돌|봄| 부담|이| 커|짐|.
|-| 청|년| 실|업|·|일|자리| 질| 저|하|:| 취|업|난|과| 비|정|규|직|·|청|년|층|의| 불|안|정|한| 고|용|(|계|약|직|·|프|리|랜|서|·|인|턴| 등|)이| 결|혼|·|출|산|·|소|비| 위|축|으로| 연결|됨|.
|-| 주|택|·|부|동|산| 문제|:| 집|값|과| 전|월|세| 부담| 상승|,| 투|기|·|가격| 불|안|정|으로| 젊|은|층|의| 주|거| 불|안|이| 심|함|.
|-| 소|득|·|자|산| 불|평|등|:| 근|로|소|득| 격|차|와| 자|산|(|부|동|산|)|으로| 인|한| 세|대|·|계|층| 간| 불|평|등|이| 심|화|됨|.
|-| 과|도|한| 교육| 경쟁|과| 사|교육| 비용|:| 입|시| 중심|의| 교육|문화|가| 스트|레스|와| 사|교육|비| 부담|을| 키|움|.
|-| 노동| 문화|(|과|로|·|워|라|밸| 부|재|):| 장|시간| 노동|,| 야|근| 문화|로| 삶|의| 질| 저|하|와| 출|산|·|가|족| 형|성|에도| 악|영|향|.
|-| 성|평|등| 부족|과| 성|차|별| 문제|:| 직|장| 내| 유|리|천|장|,| 성|폭|력|·|성|희|롱|,| 전|통|적| 가|부|장|적| 역할| 기대| 등|으로| 여성|의| 사회|·|경제| 활동|에| 제|약|.
|-| 정신|건|강| 문제|·|높|은| 자|살|률|:|

In [ ]:
messages = [
  SystemMessage(content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
  HumanMessage(content="부산의 현재 시각은 몇 시인가요?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

is_first = True

for chunk in response:
  print("chunk type: ", type(chunk))
  if is_first:
    is_first = False
    gathered = chunk
  else:
    gathered += chunk
    
  print("content: ", gathered.content, "tool_call_chunk", gathered.tool_calls)
  
messages.append(gathered)
